## Загрузим нужные библиотеки

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import tree
import numpy as np
from sklearn.metrics import recall_score, precision_score

%matplotlib inline

In [ ]:
df = pd.read_csv("https://vk.com/doc3811482_638349011?hash=sAyvkMFNDVnxNZUMeyP7GkdvJkMHMukqQuOkA2uZfkP&dl=k8J7fxKGaYY6V93yMCjmtwBBM3xcHSB2zxHXk4Ue7Oo")
df_test = pd.read_csv('https://vk.com/doc3811482_638349002?hash=CNhvkBU5xHnUMlsYHZy6DH6KoIZGQWdojFFkzCHxGGw&dl=rOYSmMLMTZhCe2tZ8lGocI2B7lMssZnBgO5qOunmKtH')

## Рассмотрим датасет по ближе

In [ ]:
df.month_id = pd.to_datetime(df.month_id).values.astype(np.int64) // 10 ** 9
df_test.month_id = pd.to_datetime(df_test.month_id).values.astype(np.int64) // 10 ** 9
df.carts_created_at = pd.to_datetime(df.carts_created_at).values.astype(np.int64) // 10 ** 9
df_test.carts_created_at = pd.to_datetime(df_test.carts_created_at).values.astype(np.int64) // 10 ** 9
df['dlitelnost'] = df.month_id - df.carts_created_at
df_test['dlitelnost'] = df_test.month_id - df_test.carts_created_at
df_test.dlitelnost

In [ ]:
df.ABC = df.ABC.astype('category')
df.ABC = df.ABC.cat.codes
df.ABC = df.ABC.astype('int')
df_test.ABC = df_test.ABC.astype('category')
df_test.ABC = df_test.ABC.cat.codes
df_test.ABC = df_test.ABC.astype('int')
df_test.ABC.value_counts()

# df = pd.concat([df, pd.get_dummies(df.ABC)])
# df_test = pd.concat([df_test, pd.get_dummies(df_test.ABC)])

In [ ]:
df.ABC.value_counts()

In [ ]:
df.communication_type = df.communication_type.astype('category')
df.communication_type = df.communication_type.cat.codes
df.communication_type = df.communication_type.astype('int')
df_test.communication_type = df_test.communication_type.astype('category')
df_test.communication_type = df_test.communication_type.cat.codes
df_test.communication_type = df_test.communication_type.astype('int')
df_test.communication_type.value_counts()

In [ ]:
df.communication_type.value_counts()

In [ ]:
df.promo = df.promo.apply(lambda x: 1 if x == '+' else 0)
df_test.promo = df_test.promo.apply(lambda x: 1 if x == '+' else 0)
df.promo

In [ ]:
df.info()

In [ ]:
df.city= df.city.fillna(0)
df.city = df.city.apply(lambda x: 4 if x == 'Москва' else x)
df.city = df.city.apply(lambda x: 3 if x == 'Санкт-Петербург' else x)
df.city = df.city.apply(lambda x: 2 if x == 'Екатеринбург' else x)
df.city = df.city.apply(lambda x: 1 if x == 'Нижний Новгород' else x)
df.city = df.city.apply(lambda x: 5 if x != 0 and x != 1 and x != 2 and x != 3 and x != 4 else x)

df_test.city= df_test.city.fillna(0)
df_test.city = df_test.city.apply(lambda x: 4 if x == 'Москва' else x)
df_test.city = df_test.city.apply(lambda x: 3 if x == 'Санкт-Петербург' else x)
df_test.city = df_test.city.apply(lambda x: 2 if x == 'Екатеринбург' else x)
df_test.city = df_test.city.apply(lambda x: 1 if x == 'Нижний Новгород' else x)
df_test.city = df_test.city.apply(lambda x: 5 if x != 0 and x != 1 and x != 2 and x != 3 and x != 4 else x)

df.city

In [ ]:
df.os = df.os.astype('category')
df.os = df.os.cat.codes
df.os = df.os.astype('int')
# df.os = df.os.fillna(-2)
df_test.os = df_test.os.astype('category')
df_test.os = df_test.os.cat.codes
df_test.os = df_test.os.astype('int')
# df_test.os = df_test.os.fillna(-2)
df_test.os.value_counts()

In [ ]:
df.browser = df.browser.astype('category')
df.browser = df.browser.cat.codes
df.browser = df.browser.astype('int')
df_test.browser = df_test.browser.astype('category')
df_test.browser = df_test.browser.cat.codes
df_test.browser = df_test.browser.astype('int')
df_test.browser.value_counts()   

In [ ]:
df.platform = df.platform.astype('category')
df.platform = df.platform.cat.codes
df.platform = df.platform.astype('int')

df_test.platform = df_test.platform.astype('category')
df_test.platform = df_test.platform.cat.codes
df_test.platform = df_test.platform.astype('int')

df.platform.value_counts()   

In [ ]:
df.platform[191:211]

In [ ]:
df.country.value_counts()

In [ ]:
df.country = df.country.fillna(0) 
df.country = df.country.apply(lambda x: 2 if x == 'Россия' or x == 'РФ' else x)
df.country = df.country.apply(lambda x: 1 if x != 2 and x != 0 else x)
df_test.country = df_test.country.fillna(0) 
df_test.country = df_test.country.apply(lambda x: 2 if x == 'Россия' or x == 'РФ' else x)
df_test.country = df_test.country.apply(lambda x: 1 if x != 2 and x != 0 else x)

df.country.value_counts()

In [ ]:
df = pd.concat([df,df_test])

cel = df.program_id.unique()[df.program_id.value_counts() > 40]  #при 40 самый лучший результат пока что
df.program_id = df.program_id.apply(lambda x: x if x in cel else 0)
df.program_id

df = pd.concat([df, pd.get_dummies(df.program_id, prefix = 'PR_id')], axis=1)
# df_test = pd.concat([df_test, pd.get_dummies(df_test.program_id, prefix='PR_id')], axis=1)
df_test = df[200000:]
df = df[0:200000]


Заменим все пропуски в данных

In [ ]:
df.gender = df.gender.fillna(-1) 
sr = df.age_indicator.mean()
df.age_indicator = df.age_indicator.fillna(-1) 
df = df.fillna(0)
df_test = df_test.fillna(0)   #Здесь, чтобы не путать, можно пробовать -1 - не работает

Пострим на распределение данных

In [ ]:
sns.countplot(x = "target" , data  = df).set_title('Распределение предсказываемой величены')

In [ ]:
df["age_indicator"].hist(bins = 40).set_title('Распределение возраста')

Взглянем на несколько величин сразу

In [ ]:
pd.set_option('display.max_columns', None)

df[5:10]



In [ ]:
features = list(set(['age_indicator',
       'carts_created_at', 'spent_time_total', 'spent_time_to_complete_hw',
       'completed_hw', 'failed_hw', 'reworked_hw', 'interacted_hw',
       'avg_hw_mark', 'test_with_good_mark', 'test_with_great_mark',
       'webinars', 'avg_quiz_result', 'notes', 'hw_leader', 'lessons',
       'activity', 'bought_d1', 'bought_d2', 'bought_d3', 'bought_d4',
       'bought_d5', 'bought_avg_duration','dlitelnost','country']))

_ = df[features].hist(figsize=(20,12))

Обратим внимание на часть столбцов с постоянными значениями

In [ ]:
df.spent_time_to_complete_hw.value_counts()

In [ ]:
df.bought_d4.value_counts()

In [ ]:
plt.rcParams['figure.figsize']=(15,15)

df_corr = df[["target",'age_indicator','month_id',
       'carts_created_at', 'spent_time_total',
       'completed_hw', 'failed_hw', 'reworked_hw', 'interacted_hw',
       'avg_hw_mark', 'test_with_good_mark', 'test_with_great_mark',
       'webinars', 'avg_quiz_result', 'promo', 'price',
       'communication_type', 'notes', 'hw_leader', 'lessons',
       'dlitelnost','city','os','browser']]

g = sns.heatmap(df_corr.corr(), square = True, annot=True)

## Выделим выборки

In [ ]:
df.dtypes[df.dtypes == "object"].values
df_test.dtypes[df_test.dtypes == "object"].values

Поместим все столбцы object в массив, для их дальнейшего удаления

In [ ]:
mass_object = df.dtypes[df.dtypes == "object"].index.values
mass_object_test = df_test.dtypes[df_test.dtypes == "object"].index.values

In [ ]:
mass_object = np.append(mass_object, "target")
mass_object_test = np.append(mass_object_test, "target")


In [ ]:
X = df.drop(mass_object, axis = 1)
# X = X.drop(['id','student_id'], axis = 1)
df_test = df_test.drop(mass_object_test, axis = 1)  
# df_test1 = df_test.drop(['id','student_id'], axis = 1)
y = df[["target"]]

In [ ]:
X.shape


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=42)
print(X_train.shape)

## Обучение модели

### regr = RandomForestClassifier(n_jobs=-1, random_state=5)
#parametres = {'criterion': ['entropy'], 'n_estimators': [250], 'max_depth': [35], 'min_samples_leaf': [1], 'max_features': [180]}   50 - 200
parametres = {'criterion': ['entropy'], 'n_estimators': [10], 'max_depth': [35], 'min_samples_leaf': [1], 'max_features': [200]}  #идеально - 180 (на цель - 50)    
RegrCV = GridSearchCV(regr, parametres, cv = 5)
RegrCV.fit(X_train, y_train.values.ravel())
print(RegrCV.best_estimator_)
clf = RegrCV.best_estimator_
RegrCV.score(X_test, y_test.values.ravel())
#0.9601666666666666
#0.97165
#cel 50 - лучше 20 и 100. Где-то здесь максимум.
#0.972 на cel 40

#0.96305 - c програм 0.9585 - без програм 0.96255 - c новой темой #0.96

In [ ]:
regr = RandomForestClassifier(n_jobs=-1, random_state=5)
parametres = {'criterion': ['entropy'], 'n_estimators': [500], 'max_depth': [35], 'min_samples_leaf': [1], 'max_features': [200]} #идеально - 180 (на цель - 50)
RegrCV = GridSearchCV(regr, parametres, cv = 5) 
RegrCV.fit(X_train, y_train.values.ravel()) 
print(RegrCV.best_estimator_) 
clf = RegrCV.best_estimator_ 
RegrCV.score(X_test, y_test.values.ravel()) 


In [ ]:
clf.feature_importances_

features = X.columns
importances = clf.feature_importances_[0:20]
indices = np.argsort(importances)

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

## Оценка точности

In [ ]:
pred = clf.predict(X_test)

In [ ]:
y_test.head(5)

In [ ]:
print('Recall: ', recall_score(y_test, pred, average='macro'))
print('Precision: ', precision_score(y_test, pred, average='macro'))
0.2* recall_score(y_test, pred, average='macro') + 0.8* precision_score(y_test, pred, average='macro')


In [ ]:
predproba = clf.predict_proba(X_test)
predproba

# Прогноз

In [ ]:
df_test

In [ ]:
PredT= clf.predict(df_test)
PredT = pd.DataFrame(PredT, columns=['target'])
PredT['id'] = df_test.id
neworder = ['id','target']
PredT=PredT.reindex(columns=neworder)
PredT


In [ ]:
PredT.target

In [ ]:
output3 = PredT #верные названия для сдачи работы
# output3.insert('id,target', predT.index)
#output3 = output3.set_index('document_id')
output3.to_csv('subnetology.csv', index=False) #, index=False
output3.head(5)